In [ ]:
import pandas as pd
from os.path import join 
from os import getcwd
import sys
raw_data_dir = "/home/davina/Private/dialysis-data"
vitals_file = "Flowsheet_Vitals_19-000093_10082020.txt"
outcome_file = "CRRT Deidentified 2017-2019.csv"

sys.path.insert(0, join(getcwd(), "../module_code"))
from data.utils import read_files_and_combine
from hcuppy.ccs import CCSEngine
from hcuppy.cpt import CPT
from data.longitudinal_utils import TIME_BEFORE_START_DATE
from data.longitudinal_utils import (
    aggregate_cat_feature,
    aggregate_ctn_feature,
    hcuppy_map_code,
    time_window_mask,
    AGGREGATE_FUNCTIONS,
)
from data.longitudinal_features import (
    load_diagnoses,
    load_vitals,
    load_labs,
    load_medications,
    load_problems,
    load_procedures,
    split_sbp_and_dbp,
)
from data.load import load_outcomes, load_static_features

In [ ]:
time_interval=None
time_before_start_date = TIME_BEFORE_START_DATE
time_window_end = "End Date"
time_interval="1D"

In [ ]:
outcomes_df = load_outcomes(raw_data_dir)


In [ ]:
static_df = load_static_features(raw_data_dir)
longitudinal_dfs = [
load_diagnoses(
	outcomes_df,
	raw_data_dir,
	time_interval=time_interval,
	time_before_start_date=time_before_start_date,
	time_window_end=time_window_end,
),
load_vitals(
	outcomes_df,
	raw_data_dir,
	time_interval=time_interval,
	time_before_start_date=time_before_start_date,
	time_window_end=time_window_end,
)
# load_medications(
# 	outcomes_df,
# 	raw_data_dir,
# 	time_interval=time_interval,
# 	time_before_start_date=time_before_start_date,
# 	time_window_end=time_window_end,
# ),
# load_labs(
# 	outcomes_df,
# 	raw_data_dir,
# 	time_interval=time_interval,
# 	time_before_start_date=time_before_start_date,
# 	time_window_end=time_window_end,
# ),
# load_problems(
# 	outcomes_df,
# 	raw_data_dir,
# 	time_interval=time_interval,
# 	time_before_start_date=time_before_start_date,
# 	time_window_end=time_window_end,
# ),
# load_procedures(
# 	outcomes_df,
# 	raw_data_dir,
# 	time_interval=time_interval,
# 	time_before_start_date=time_before_start_date,
# 	time_window_end=time_window_end,
# ),
]

# outer join features with each other so patients who might not have allergies,  for example, are still included
# from functools import reduce
# features = reduce(
# lambda df1, df2: pd.merge(df1, df2, on="IP_PATIENT_ID", how="outer"),
# longitudinal_dfs + [static_df],
# )
# # inner join features with outcomes (only patients with outcomes)
# features_with_outcomes = pd.merge(
# features, outcomes_df, on="IP_PATIENT_ID", how="inner"
# )


In [ ]:
longitudinal_dfs[1]

In [ ]:
from functools import reduce
merge_on = ["IP_PATIENT_ID", "DATE"] if time_interval else "IP_PATIENT_ID"
features = reduce(
        lambda df1, df2: pd.merge(df1, df2, on=merge_on, how="outer"),
        longitudinal_dfs
    )


In [ ]:
print(features.index.has_duplicates)
features

In [ ]:
features_with_static = features.join(static_df, how="outer")
# pd.concat([features, static_df], axis=1, join="outer") ## different id order than join
print(features_with_static.index.has_duplicates)
features_with_static

In [ ]:
working_outcomes_df = (
    outcomes_df.dropna(subset=["IP_PATIENT_ID"])
    .drop_duplicates(subset=["IP_PATIENT_ID"], keep="first")
    .set_index("IP_PATIENT_ID")
)
# inner join features with outcomes (only patients with outcomes)
# merge is incorrect here for the same reason as static
features_with_outcomes = features.join(working_outcomes_df, how="inner")
# features.join(working_outcomes_df, how="inner")

In [ ]:
print(features_with_outcomes.index.has_duplicates)
features_with_outcomes

In [ ]:
features_with_outcomes.index.value_counts()